In [1]:
import numpy as np
import sys
#sys.path.append('/home/yungdankblast/Desktop/Research/delay-optimizer')
sys.path.append('/home/cayjobla/Desktop/Research/delay-optimizer')
from DelayEffects.Analyzer_new import FuncOpt
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
from matplotlib.lines import Line2D
from Optimizer_Scripts.DelayTypeGenerators import *

# <center> Time-Loss Plots

In [2]:
# Initialize
dimensions = [2, 10, 100, 1000]
ack = {d:FuncOpt('Ackley', d) for d in dimensions}
rast = {d:FuncOpt('Rastrigin', d) for d in dimensions}
rosen = {d:FuncOpt('Rosenbrock', d) for d in dimensions}
zak = {d:FuncOpt('Zakharov', d) for d in dimensions}

## Helper Functions

In [3]:
def load_vals(func_dict, file_tags):
    if type(file_tags) == str:
        file_tags = {d:file_tags for d in dimensions}
    loss_name = func_dict[dimensions[0]].loss_name
    filenames = {d:f'Data/{loss_name}{d}d_{file_tags[d]}.npz' for d in dimensions}
    for d, func_obj in func_dict.items():
        func_obj.load(filenames[d])          # Load data into object
        
def unload(func_dict):
    for d, func_obj in func_dict.items():
        func_obj.reset()

In [ ]:
def init_time_loss_axes(axes=None):
    if axes is None:
        fig, axes = plt.subplots(2, 2, figsize=(15,10), dpi=400)
    return plt.gcf(), axes.ravel()

def format_plot(axes, line_objs, labels, xlabel, ylabel, ylim=None, logscale=False, **kwargs):
    for ax in axes.ravel():
        ax.legend(line_objs, labels, **kwargs)
        ax.set_xlabel(xlabel, fontsize=12)
        ax.set_ylabel(ylabel, fontsize=12)
        if ylim is not None:
            ax.set_ylim(ylim)
        if logscale is True:
            ax.set_yscale('log')

def plot_time_loss(func_dict, axes=None, logscale=False, alpha=1/255, **kwargs):
    fig, axes = init_time_loss_axes(axes)
    for i, (n, func_obj) in enumerate(func_dict.items()):
        loss_arr = func_obj.get_loss_array()
        for j in range(len(func_obj.x_inits)):               # Plot
            axes[i].plot(np.arange(func_obj.maxiter+1), loss_arr[j], alpha=alpha, **kwargs)
        
        axes[i].set_xlim([0, func_obj.maxiter])
        axes[i].set_title(f"{func_obj.loss_name} {n}d", fontsize=16)
            
    return axes

def plot_mean_time_loss(func_dict, axes=None, **kwargs):
    fig, axes = init_time_loss_axes(axes)
    for i, (n, func_obj) in enumerate(func_dict.items()):
        loss_arr = func_obj.get_loss_array()
        axes[i].plot(np.arange(func_obj.maxiter+1), np.mean(loss_arr, axis=0), **kwargs)
        
    return axes

def quick_plot_time_loss(func_dict, param_type, delay_list):
    # Init
    axes = None
    colors = ['b', 'r']
    mean_colors = ['navy', 'darkred']
    label_lines = [Line2D([0], [0], c=colors[0]), Line2D([0], [0], lw=2, c=mean_colors[0]),
                   Line2D([0], [0], c=colors[1]), Line2D([0], [0], lw=2, c=mean_colors[1])]

    for k, delay_type in enumerate(delay_list):
        load_vals(func_dict, delay_type.name, param_type)
        print([func_dict[d].lr_params for d in dimensions])
        axes = plot_time_loss(func_dict, logscale=False, axes=axes, c=colors[k])
        axes = plot_mean_time_loss(func_dict, axes=axes, c=mean_colors[k])

    for i, ax in enumerate(axes):
        ax.legend(label_lines, ["Undelayed", "Mean Undelayed", "Delayed", "Mean Delayed"], loc='lower left')
        ax.set_xlabel("Step", fontsize=12)
        ax.set_ylabel("Loss", fontsize=12)
        ax.set_ylim([0, 22])

    plt.show()

In [ ]:
def calc_plot(func_dict, param_type, delay_list):
    axes = None
    colors = ['b', 'r']
    mean_colors = ['navy', 'darkred']
    label_lines = [Line2D([0], [0], c=colors[0]), Line2D([0], [0], lw=2, c=mean_colors[0]),
                   Line2D([0], [0], c=colors[1]), Line2D([0], [0], lw=2, c=mean_colors[1])]

    for k, delay_type in enumerate(delay_list):
        for d, func_obj in func_dict.items():
            func_obj.initialize_points('random', 50)
            func_obj.optimize(delay_type, param_type=param_type, const_lr=False)
            print(func_obj.lr_params)
        axes = plot_time_loss(func_dict, logscale=False, axes=axes, c=colors[k])
        axes = plot_mean_time_loss(func_dict, axes=axes, c=mean_colors[k])

    for i, ax in enumerate(axes):
        ax.legend(label_lines, ["Undelayed", "Mean Undelayed", "Delayed", "Mean Delayed"], loc='lower left')
        ax.set_xlabel("Step", fontsize=16)
        ax.set_ylabel("Loss", fontsize=16)
        ax.set_ylim([0, 22])
    
    plt.savefig("Plots/ack_optimal_time_loss.png")
    plt.show()

## Ackley

In [ ]:
fig, axes = init_time_loss_axes()

load_vals(ack, "undel_const_1.5")
axes = plot_time_loss(ack, c='g', axes=axes, alpha=1/80)
axes = plot_mean_time_loss(ack, c='b', lw=3, axes=axes)

load_vals(ack, "stoch_const_1.5")
axes = plot_time_loss(ack, c='orange', axes=axes, alpha=1/170)
axes = plot_mean_time_loss(ack, c='r', lw=3, axes=axes)

line_objs = [Line2D([0], [0], c='g'), Line2D([0], [0], lw=2, c='b'),
             Line2D([0], [0], c='orange'), Line2D([0], [0], lw=2, c='r')]
labels = ["Undelayed", "Mean Undelayed", "Delayed", "Mean Delayed"]
format_plot(axes, line_objs, labels, loc='lower left', xlabel="Step", ylabel="Loss", ylim=[0,22])
plt.tight_layout()
plt.savefig('Plots/Ackley_time_loss_const_1.5.png', bbox_inches="tight")
plt.show()

In [ ]:
fig, axes = init_time_loss_axes()

load_vals(ack, "undel_adapt_opt")
axes = plot_time_loss(ack, c='g', axes=axes, alpha=1/80)
axes = plot_mean_time_loss(ack, c='b', lw=3, axes=axes)

load_vals(ack, "stoch_adapt_opt")
axes = plot_time_loss(ack, c='orange', axes=axes, alpha=1/170)
axes = plot_mean_time_loss(ack, c='r', lw=3, axes=axes)

line_objs = [Line2D([0], [0], c='g'), Line2D([0], [0], lw=2, c='b'),
             Line2D([0], [0], c='orange'), Line2D([0], [0], lw=2, c='r')]
labels = ["Undelayed", "Mean Undelayed", "Delayed", "Mean Delayed"]
format_plot(axes, line_objs, labels, loc='upper right', xlabel="Step", ylabel="Loss", ylim=[0,22])
#plt.savefig('Plots/Ackley_time_loss_adapt_opt.png', bbox_inches="tight")
plt.show()

In [ ]:
delay_list = [Undelayed(), Stochastic(1, 1000)]
quick_plot_time_loss(ack, 'optimal', delay_list)

In [ ]:
delay_list = [Undelayed(), Stochastic(1, 2000)]

# Init
axes = None
colors = ['b', 'r', 'navy', 'darkred']
param_types = ['optimal', 'given']
label_lines = [Line2D([0], [0], c=colors[0]), Line2D([0], [0], lw=2, c=colors[len(delay_list)]),
               Line2D([0], [0], c=colors[1]), Line2D([0], [0], lw=2, c=colors[len(delay_list)+1])]

for k, delay_type in enumerate(delay_list):
    load_vals(ack, delay_type.name, param_types[k])
    axes = plot_time_loss(ack, logscale=False, axes=axes, c=colors[k])
    axes = plot_mean_time_loss(ack, axes=axes, c=mean_colors[k])

for i, ax in enumerate(axes):
    ax.legend(label_lines, ["Undelayed", "Mean Undelayed", "Delayed", "Mean Delayed"], loc='lower right')
    ax.set_xlabel("Step", fontsize=16)
    ax.set_ylabel("Loss", fontsize=16)
    ax.set_ylim([0, 22])

plt.show()

In [ ]:
delay_list = [Undelayed(), Stochastic(1, 2000)]
quick_plot_time_loss(ack, 'optimal')

In [ ]:
delay_list = [Undelayed(), Stochastic(1, 2000)]
quick_plot_time_loss(ack, 'given', delay_list)

In [ ]:
calc_plot(ack, 'optimal', [Undelayed(), Stochastic(1,1000)])

In [ ]:
func_obj = ack[100]
func_obj.load("Data/Ackley100d_undel_BensonLR.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='b', label="Undelayed Opt")
print(func_obj.lr_params)
func_obj.load("Data/Ackley100d_stoch_BensonLR.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='r', label="Stochastic Opt")
print(func_obj.lr_params)
func_obj.load("Data/Ackley100d_undel_const_1.5.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='g', label="Undelayed 1.5")
print(func_obj.lr_params)
func_obj.load("Data/Ackley100d_stoch_const_1.5.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='orange', label="Stochastic 1.5")
print(func_obj.lr_params)
plt.ylim([0,22])
plt.legend()
plt.title("Ackley 100d")
plt.show()

In [ ]:
func_obj = FuncOpt("Ackley", 100, maxiter=3000)
func_obj.load("Data/Ackley100d_undel_BensonUndel.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='b', label="Undelayed")
print(func_obj.lr_params)
func_obj.load("Data/Ackley100d_stoch_BensonUndel.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='r', label="Stochastic")
print(func_obj.lr_params)
plt.ylim([0,22])
plt.legend()
plt.title("Ackley 100d (Undelayed Params)")
plt.show()

In [ ]:
func_obj = FuncOpt("Ackley", 100, maxiter=3000)
func_obj.load("Data/Ackley100d_undel_BensonDel.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='b', label="Undelayed")
print(func_obj.lr_params)
func_obj.load("Data/Ackley100d_stoch_BensonDel.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='r', label="Stochastic")
print(func_obj.lr_params)
plt.ylim([0,22])
plt.legend()
plt.title("Ackley 100d (Delayed Params)")
plt.show()

In [ ]:
func_obj = FuncOpt("Ackley", 100, maxiter=5000)
func_obj.load("Data/Ackley100d_stoch_BensonLR_5000.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='b', label="Optimal params")
print(func_obj.lr_params)
print("Avg. Final Loss:", func_obj.get_mean_final(func_obj.loss_vals))
func_obj.load("Data/Ackley100d_stoch_const_1.5_5000.npz")
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0), c='r', label="Hand-chosen")
print(func_obj.lr_params)
print("Avg. Final Loss:", func_obj.get_mean_final(func_obj.loss_vals))
plt.ylim([0,22])
plt.legend()
plt.title("Ackley 100d")
plt.show()

In [ ]:
func_obj = ack[2]
func_obj.initialize_points('random', 500)
func_obj.optimize(Undelayed(), param_type='optimal', const_lr=False)

In [ ]:
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0))
plt.ylim([0,21])
plt.show()
print(func_obj.lr_params)

In [ ]:
plt.plot(np.arange(func_obj.maxiter+1), np.mean(func_obj.get_loss_array(), axis=0))
plt.ylim([0,21])
plt.show()
print(func_obj.lr_params)

In [ ]:
ax = plot_time_loss(plot_data, means=True, loss_name="Ackley")
ax[0].set_ylim([0,23])
ax[1].set_ylim([0,23])
ax[2].set_ylim([0,23])
ax[3].set_ylim([0,23])
plt.savefig("ack_time_loss.png")
plt.show()